In [4]:
import pandas as pd

# Your merged dataset
merged_df = pd.read_csv('../data/merged_file.csv')

# Kaggle dataset
kaggle_df = pd.read_csv('../data/final_dataset.csv')

# Convert 'Date' to datetime if needed
merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')
kaggle_df['Date'] = pd.to_datetime(kaggle_df['Date'], errors='coerce')

# Extract year
merged_df['Year'] = merged_df['Date'].dt.year
kaggle_df['Year'] = kaggle_df['Date'].dt.year

# Get unique years
merged_years = sorted(merged_df['Year'].dropna().unique())
kaggle_years = sorted(kaggle_df['Year'].dropna().unique())

print("Seasons in your merged dataset:", merged_years)
print("Seasons in Kaggle dataset:", kaggle_years)


print("Row counts per season in your merged dataset:")
print(merged_df.groupby('Year').size())

print("\nRow counts per season in Kaggle dataset:")
print(kaggle_df.groupby('Year').size())

duplicate_rows = merged_df[merged_df.duplicated(subset=['Date','HomeTeam','AwayTeam'])]
print(f"Number of duplicate match rows: {len(duplicate_rows)}")

Seasons in your merged dataset: [np.int32(2000), np.int32(2001), np.int32(2002), np.int32(2003), np.int32(2004), np.int32(2005), np.int32(2006), np.int32(2007), np.int32(2008), np.int32(2009), np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020)]
Seasons in Kaggle dataset: [np.int32(2000), np.int32(2001), np.int32(2002), np.int32(2003), np.int32(2004), np.int32(2005), np.int32(2006), np.int32(2007), np.int32(2008), np.int32(2009), np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018)]
Row counts per season in your merged dataset:
Year
2000    205
2001    373
2002    391
2003    359
2004    392
2005    374
2006    394
2007    371
2008    379
2009    378
2010    374
2011    377
2012    391
2013    372
2014    380
2015    380
2016    378
2017    401
2018    331
2019    199
2020    

C:\Users\user\AppData\Local\Temp\ipykernel_19196\3385678992.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_19196\3385678992.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  kaggle_df['Date'] = pd.to_datetime(kaggle_df['Date'], errors='coerce')


In [5]:
comparison = pd.DataFrame({
    'merged_count': merged_df[merged_df['Year'] <= 2018].groupby('Year').size(),
    'kaggle_count': kaggle_df.groupby('Year').size()
})
comparison['diff'] = comparison['merged_count'] - comparison['kaggle_count']
print(comparison)

      merged_count  kaggle_count  diff
Year                                  
2000           205           205     0
2001           373           373     0
2002           391           391     0
2003           359           359     0
2004           392           392     0
2005           374           374     0
2006           394           394     0
2007           371           371     0
2008           379           379     0
2009           378           378     0
2010           374           374     0
2011           377           377     0
2012           391           391     0
2013           372           372     0
2014           380           380     0
2015           380           380     0
2016           378           378     0
2017           401           401     0
2018           331           171   160


In [6]:
merged_2018 = merged_df[merged_df['Year'] == 2018]
kaggle_2018 = kaggle_df[kaggle_df['Year'] == 2018]

# Compare basic info
print(merged_2018[['Date','HomeTeam','AwayTeam']].drop_duplicates().shape)
print(kaggle_2018[['Date','HomeTeam','AwayTeam']].drop_duplicates().shape)


(331, 3)
(171, 3)


In [7]:
# Focus on 2018
merged_2018 = merged_df[merged_df['Year'] == 2018]
kaggle_2018 = kaggle_df[kaggle_df['Year'] == 2018]

# Find extra rows in your merged dataset (based on Date, HomeTeam, AwayTeam)
extra_rows = merged_2018.merge(
    kaggle_2018[['Date','HomeTeam','AwayTeam']],
    on=['Date','HomeTeam','AwayTeam'],
    how='left',
    indicator=True
).query('_merge == "left_only"')

print(f"Number of extra rows in merged dataset for 2018: {len(extra_rows)}")
print(extra_rows)


Number of extra rows in merged dataset for 2018: 160
          Date Div      HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG FTR  \
171 2018-10-08  E0    Man United       Leicester     2     1     1     0   H   
172 2018-11-08  E0   Bournemouth         Cardiff     2     0     1     0   H   
173 2018-11-08  E0        Fulham  Crystal Palace     0     2     0     1   A   
174 2018-11-08  E0  Huddersfield         Chelsea     0     3     0     2   A   
175 2018-11-08  E0     Newcastle       Tottenham     1     2     1     2   A   
..         ...  ..           ...             ...   ...   ...   ...   ...  ..   
326 2018-08-12  E0     Leicester       Tottenham     0     2     0     1   A   
327 2018-08-12  E0    Man United          Fulham     4     1     3     0   H   
328 2018-08-12  E0      West Ham  Crystal Palace     3     2     0     1   H   
329 2018-09-12  E0     Newcastle          Wolves     1     2     1     1   A   
330 2018-10-12  E0       Everton         Watford     2     2     1 

In [11]:
merged_df.dtypes


Date        datetime64[ns]
Div                 object
HomeTeam            object
AwayTeam            object
FTHG                 int64
FTAG                 int64
HTHG                 int64
HTAG                 int64
FTR                 object
HS                   int64
AS                   int64
HST                  int64
AST                  int64
HC                   int64
AC                   int64
HF                   int64
AF                   int64
HY                   int64
AY                   int64
HR                   int64
AR                   int64
Year                 int32
dtype: object